Text Classification Project

In [ ]:
import sys
!{sys.executable} -m pip install transformers

In [ ]:
# Necessary Code
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn.metrics import confusion_matrix
from statistics import mean
from numpy import std
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn.neighbors import KNeighborsClassifier





df = pd.read_table("text.txt",header=None)
df.columns = ['text']
df[['topic','sentiment','id','text']]= df.text.str.split(' ',n=3,expand=True)


df = df.drop_duplicates(subset=['text'], keep='first')
print(df.sentiment.value_counts())

# pre-processing
df['text'] = df['text'].str.replace(" n't ", " not ")
df['text'] = df['text'].str.replace(" 's ", "'s ")
df['text'] = df['text'].str.replace(" 'm ", "'m ")

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

contractions = {
    "n't": "not",
    "'re": "are",
    "'s": "",
    "'m": "",
    "'ve": "have",
    "'ll": "will",
    "'d": "would"
}
pattern = re.compile("|".join(contractions.keys()))

stop_words = set(nltk.corpus.stopwords.words("english"))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word.lower())
for word in word_tokenize(pattern.sub(lambda match: contractions[match.group(0)], x))
if word.lower() not in stop_words and word not in punctuation]))

# df.to_csv('/files/text_new.csv', index=False)

mapper = {"pos": 0, "neg": 1}
df['sentiment'] = df.sentiment.map(mapper)



ML MODELS

Glove

In [ ]:
import nltk
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.python.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.python.keras.models import Sequential
import tensorflow as tf
import os

from tensorflow.python.keras.regularizers import l2
def get_weight_matrix(vocab, vocab_size, embeddings_index):
    weight_matrix = np.zeros((vocab_size, 100))
    for word, i in vocab.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            weight_matrix[i] = embedding_vector

    return weight_matrix


embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('GloVe data loaded')
df = pd.read_csv('text_new.csv', encoding='utf-8')

df['sentiment'] = df['sentiment'].map({'neg': 1, 'pos': 0})

X = df.text.values
y = df['sentiment']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1

X = pad_sequences(X, maxlen=200)
embedding_vectors = get_weight_matrix(tokenizer.word_index, vocab_size, embeddings_index)
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
with mirrored_strategy.scope():
    model = Sequential()
    model.add(Embedding(vocab_size, output_dim=100, input_length=200,
                        trainable=False))
    model.set_weights([embedding_vectors])
    model.add(LSTM(units=128))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0001)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

kfold = KFold(n_splits=5, shuffle=True)
for train_indices, val_indices in kfold.split(X):
    X_train, X_val = X[train_indices], X[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]
    model.fit(X_train, y_train,
              batch_size=128,
              epochs=7,
              verbose=1,
              validation_data=(X_val, y_val))

    y_pred = (model.predict(X_val) >= 0.5).astype("int")

    score = accuracy_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)


    print(classification_report(y_val, y_pred))
    print(f'Accuracy score: {score:.4f}')
    print(f'Confusion matrix:\n{cm}')


W2V

In [ ]:
def normalize():
    X = []
    stop_words = set(nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    for par in news["text"].values:
        tmp = []
        sentences = nltk.sent_tokenize(par)
        for sent in sentences:
            sent = sent.lower()
            tokens = tokenizer.tokenize(sent)
            filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
            tmp.extend(filtered_words)
        X.append(tmp)

    return X

def get_weight_matrix(model, vocab, vocab_size):
    weight_matrix = np.zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        if word in model.wv.key_to_index:
            weight_matrix[i] = model.wv.get_vector(word)
        else:
            print(i)
    return weight_matrix


df = pd.read_csv('text_new.csv', encoding='utf-8')
df['sentiment'] = df['sentiment'].map({'neg': 1, 'pos': 0})

X = normalize()
y = df['sentiment']

w2v = gensim.models.Word2Vec(X, vector_size=100, window=10, min_count=1)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1
X = pad_sequences(X, maxlen=200)
embedding_vectors = get_weight_matrix(w2v, tokenizer.word_index, vocab_size)
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
with mirrored_strategy.scope():
    model = Sequential()
    model.add(Embedding(vocab_size, output_dim=100, input_length=200,
                        trainable=False))
    model.set_weights([embedding_vectors])
    model.add(LSTM(units=128))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.00001)))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

kfold = KFold(n_splits=5, shuffle=True)
for train_indices, val_indices in kfold.split(X):
    X_train, X_val = X[train_indices], X[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]
    model.fit(X_train, y_train,
              batch_size=128,
              epochs=8,
              verbose=1,
              validation_data=(X_val, y_val))

    y_pred = (model.predict(X_val) >= 0.5).astype("int")

    # Calculate the accuracy score
    score = accuracy_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)

    print(classification_report(y_val, y_pred))

    # Print the accuracy score
    print(f'Accuracy score: {score:.4f}')
    print(f'Confusion matrix:\n{cm}')



Logistic Regression

In [ ]:
X = df.text.values.astype("str")
X = TfidfVectorizer().fit_transform(X)

y = df.sentiment.values

model = LogisticRegression()

for k in range(2,15):
  cv = KFold(n_splits=k, random_state=1, shuffle=True)
  scoreF1 = cross_val_score(model, X, y, scoring="f1", cv=cv, n_jobs=-1)
  scoreRecall = cross_val_score(model, X, y, scoring="recall", cv=cv, n_jobs=-1)
  scorePrecision = cross_val_score(model, X, y, scoring="precision", cv=cv, n_jobs=-1)
  scoreAccuracy = cross_val_score(model, X, y, scoring="accuracy", cv=cv, n_jobs=-1)

  print('F1 for k %d: %.3f (%.3f)' % (k, mean(scoreF1), std(scoreF1)))
  print('Recall for k %d: %.3f (%.3f)' % (k, mean(scoreRecall), std(scoreRecall)))
  print('Precision for k %d: %.3f (%.3f)' % (k, mean(scorePrecision), std(scorePrecision)))
  print('Accuracy for k %d: %.3f (%.3f)' % (k, mean(scoreAccuracy), std(scoreAccuracy)))
  # Use cross_val_predict to make predictions for each fold of the cross-validation
  y_pred = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)

  # Compute the confusion matrix for the combined predictions
  cm = confusion_matrix(y, y_pred)

  print(cm)
  print()


Naive Bayes

In [ ]:

# Read csv file
news=pd.read_csv('text_new.csv')
news['sentiment'].replace(['neg', 'pos'], [0,1], inplace=True)
x_train, x_test, y_train, y_test = train_test_split(news["text"], news["sentiment"], test_size=0.2, stratify=news["sentiment"], random_state=1)
news.head()

# Pre-processing & pipeline
model_nb = Pipeline([
    ("tf-idf",TfidfVectorizer(stop_words="english")),
    ("clf",MultinomialNB())
])

# k = 5-fold cross-validation
scores_nb = cross_validate(model_nb, x_train, y_train, cv=5, scoring=("accuracy", "f1", "recall", "precision"), n_jobs=5)

# train the model on the entire training set
model_nb.fit(x_train, y_train)

print("Mean accuracy:", scores_nb["test_accuracy"].mean())
print("Accuracy standard deviation:", scores_nb["test_accuracy"].std())
print("Mean f1 score:", scores_nb["test_f1"].mean())
print("Mean recall score:", scores_nb["test_recall"].mean())
print("Mean precision score:", scores_nb["test_precision"].mean())
# print("conusion matrix : ", confusion_matrix(y_test,scores_nb) )

KNN

In [ ]:
# Read csv file

scores_knn = cross_validate(model_knn, x_train, y_train, cv=5, scoring=("accuracy", "f1", "recall", "precision"))

error_rate = []


for i in range(1,100):
    model_knn = Pipeline([
        ("tf-idf", TfidfVectorizer(stop_words="english")),
        ("clf", KNeighborsClassifier(n_neighbors=i))
    ])
    model_knn.fit(x_train, y_train)
    y_test_predicted = model_knn.predict(x_test)
    error_rate.append(np.mean(y_test_predicted != y_test))

plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')


news=pd.read_csv('text_new.csv')
news['sentiment'].replace(['neg', 'pos'], [0,1], inplace=True)
x_train, x_test, y_train, y_test = train_test_split(news["text"], news["sentiment"], test_size=0.2, stratify=news["sentiment"], random_state=1)
news.head()

# Pre-processing & pipeline
model_KNN = Pipeline([
    ("tf-idf",TfidfVectorizer(stop_words="english")),
    ("clf", KNeighborsClassifier())
])

# k = 5-fold cross-validation
for k in range(2,15):
  scores_KNN = cross_validate(model_KNN, x_train, y_train, cv=k, scoring=("accuracy", "f1", "recall", "precision"), n_jobs=5)

  # train the model on the entire training set
  model_KNN.fit(x_train, y_train)
  print("KFOLD-%d"%k)
  print("Mean accuracy:", scores_KNN["test_accuracy"].mean())
  print("Accuracy standard deviation:", scores_KNN["test_accuracy"].std())
  print("Mean f1 score:", scores_KNN["test_f1"].mean())
  print("Mean recall score:", scores_KNN["test_recall"].mean())
  print("Mean precision score:", scores_KNN["test_precision"].mean())
  print()

In [ ]:
# Duplicate IDs
id_counts = df['id'].value_counts()
duplicate_ids = id_counts[id_counts > 1]
duplicate_id_list = []
max_occurence = (None,0)
min_occurence = (None,10**5)
freq = {}
for id, count in zip(duplicate_ids.index, duplicate_ids.values):
    duplicate_id_list.append((id, count))
    if count > max_occurence[1]:
      max_occurence = (id,count)
    if count < min_occurence[1]:
      min_occurence = (id,count)
    if count in freq:
      freq[count] +=1
    else:
      freq[count] = 1

print("Amount of IDs with duplicates: %d" % len(duplicate_id_list))
print("ID with most duplicates: %s-%d" % max_occurence) #582.txt
print("ID with fewest duplicates: %s-%d" % min_occurence) # 957.txt
# for duplicates, count in freq.items():
#   print("This amount of duplicates - %d - occurs %d times" % (duplicates,count))

id_mask = df.id == '582.txt'
id_rows = df.loc[id_mask]
print(id_rows)

# Duplicate text

duplicates_id_text = df.duplicated(subset=['id', 'text'], keep=False)
num_duplicates = duplicates_id_text.sum()
print(f'There are {num_duplicates} rows with the same ID and text.')


duplicates_text = df.duplicated(subset=['text'], keep=False)
num_duplicates = duplicates_text.sum()
print(f'There are {num_duplicates} rows with the same text.')
duplicate_text = df[duplicates_text]

print(duplicate_text.sort_values(by=['text']))

cleaned = df.drop_duplicates(subset=['text'], keep='first')
duplicates_text = cleaned.duplicated(subset=['text'], keep=False)
num_duplicates = duplicates_text.sum()
print(f'There are {num_duplicates} rows with the same text.')


# Task 7: Draw Wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Source for tutorial: https://www.the-analytics.club/word-cloud-python-example

# Step 1: Separate dataframe into a pos and neg dataframe
pos_df = df[df['sentiment'] == 'pos']
neg_df = df[df['sentiment'] == 'neg']

# Step 2: Concatenate all positive and negative reviews into 1 string (all_pos_text, all_neg_text)
all_pos_text = pos_df['text'].str.cat(sep=' ')
all_neg_text = neg_df['text'].str.cat(sep=' ')

# Step 2.5: Remove all words that are smaller than 3 characters
# Use the method spilt to get each word, then use list comprehension to append new word into new list
all_pos_text_list = [word for word in all_pos_text.split() if len(word) > 3]
all_neg_text_list = [word for word in all_neg_text.split() if len(word) > 3]

# Use join method to convert list into a single string that will be fed to the WordCloud
# Note that ' ' separates the words in the string
all_pos_text_cleaned = ' '.join(all_pos_text_list)
all_neg_text_cleaned = ' '.join(all_neg_text_list)

# Step 3: Generate wordcloud
pos_wordcloud = WordCloud().generate(all_pos_text_cleaned)
neg_wordcloud = WordCloud().generate(all_neg_text_cleaned)


# Step 4: Display generated wordclouds
plt.imshow(pos_wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Positive Reviews', color = 'green', fontdict = {'fontsize': 24}) # Red green for positive review & Make font larger
plt.figure(figsize=(380,380))
plt.show()

plt.imshow(neg_wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.title('Negative Reviews', color = 'red', fontdict = {'fontsize': 24}) # Red colour for negative review & Make font larger
plt.figure(figsize=(380,380))
plt.show()

import re

df['text_len_chars'] = df['text'].str.len()
df['text_len_words'] = df['text'].apply(lambda x: len(re.findall(r'\b\w+\b', x)))

print("Length of texts segment: \n")
# calculate the average length of texts in characters and words
avg_len_chars = df['text_len_chars'].mean()
avg_len_words = df['text_len_words'].mean()
print("Average length of texts (in characters): {:.2f}".format(avg_len_chars))
print("Average length of texts (in words): {:.2f} ".format(avg_len_words))
print()

# calculate the average word count for positive and negative reviews
avg_word_count_pos = df.loc[df['sentiment'] == 'pos', 'text_len_words'].mean()
avg_word_count_neg = df.loc[df['sentiment'] == 'neg', 'text_len_words'].mean()

print("Average word count for positive reviews:", round(avg_word_count_pos, 2))
print("Average word count for negative reviews:", round(avg_word_count_neg, 2))
print()

# calculate the average word length of texts for each category
avg_word_len_by_category = df.groupby('topic')['text_len_words'].mean()
print("Average word length of texts for each category:")
print(avg_word_len_by_category)
print()

# calculate the variance of text lengths in characters and words
var_words = df['text_len_words'].var()
print("Variance of text lengths (in words): {:.2f}".format(var_words))
print()

# find the smallest and longest texts
min_idx = df['text_len_chars'].idxmin()
max_idx = df['text_len_chars'].idxmax()

smallest_text = df.loc[min_idx, 'text']
smallest_text_topic = df.loc[min_idx, 'topic']
smallest_text_sentiment = df.loc[min_idx, 'sentiment']
smallest_text_word_len = df.loc[min_idx, 'text_len_words']

longest_text = df.loc[max_idx, 'text']
longest_text_topic = df.loc[max_idx, 'topic']
longest_text_sentiment = df.loc[max_idx, 'sentiment']
longest_text_word_len = df.loc[max_idx, 'text_len_words']

print("Smallest text:")
print("Topic:", smallest_text_topic)
print("Sentiment:", smallest_text_sentiment)
print("Text:", smallest_text)
print("Word length:", smallest_text_word_len)
print("\nLongest text:")
print("Topic:", longest_text_topic)
print("Sentiment:", longest_text_sentiment)
print("Text:", longest_text)
print("Word length:", longest_text_word_len)


###### SENTIMENT CLASSES DISTRIBUTION
camera_sentiment_dist=df[df["topic"] == "camera"].sentiment.value_counts()
music_sentiment_dist=df[df["topic"] == "music"].sentiment.value_counts()
health_sentiment_dist=df[df["topic"] == "health"].sentiment.value_counts()
dvd_sentiment_dist=df[df["topic"] == "dvd"].sentiment.value_counts()
books_sentiment_dist=df[df["topic"] == "books"].sentiment.value_counts()
software_sentiment_dist=df[df["topic"] == "software"].sentiment.value_counts()

fig, axes = plt.subplots(nrows=2, ncols=3)

camera_sentiment_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12), title="camera", ax=axes[0,0])
music_sentiment_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12), title="music", ax=axes[0,1])
health_sentiment_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12), title="health", ax=axes[0,2])
dvd_sentiment_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12), title="dvd", ax=axes[1,0])
books_sentiment_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12), title="books", ax=axes[1,1])
software_sentiment_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12), title="software", ax=axes[1,2])

############## BINARIZE DATA
neg_df=df[df["sentiment"] == "neg"]
pos_df=df[df["sentiment"] == "pos"]

################ LABEL ENCODE
# df["topic"] = df["topic"].astype(int)




In [ ]:
# Task 1
# Show an example from the dataset
# read it as table, read_csv breaks it.
df = pd.read_table("text.txt",header=None)
# extract an example entry
print(df.head(1))
# Task 2
# Transform the data into a form that we can work with
# set a column for the dataframe
df.columns = ['text']
# split the text into columns
df[['topic','sentiment','id','text']]= df.text.str.split(' ',n=3,expand=True)
print(df.head(3))
# Task 3
# Get the distribution of positive and negative reviews
print(df.sentiment.value_counts())
# Task 4: How about the distribution of topics? Are the topics normally distributed?
topic_dist=df.topic.value_counts()
topic_dist.plot(kind='pie', autopct='%1.0f%%', figsize=(15,12))
# Task 5: How about the distribution of sentiment among topics?
for topic in df.topic.unique():
  print("Topic %s has %d negative reviews" % (topic,len(df[(df['topic']==str(topic)) & (df['sentiment']=='neg')])))
  print("Topic %s has %d positive reviews" % (topic,len(df[(df['topic']==str(topic)) & (df['sentiment']=='pos')])))
  # Task 6: How about the length of texts? Are the texts in similar length? Any large or short sentences that you can drop from your study?
import re

df['text_len_chars'] = df['text'].str.len()
df['text_len_words'] = df['text'].apply(lambda x: len(re.findall(r'\b\w+\b', x)))

# calculate the average length of texts in characters and words
avg_len_chars = df['text_len_chars'].mean()
avg_len_words = df['text_len_words'].mean()
print("Average length of texts (in characters): {:.2f}".format(avg_len_chars))
print("Average length of texts (in words): {:.2f} ".format(avg_len_words))
print("")
# calculate the average word length of texts for each category
avg_word_len_by_category = df.groupby('topic')['text_len_words'].mean()
print("Average word length of texts for each category:")
print(avg_word_len_by_category)
print()

# calculate the variance of text lengths in characters and words
var_words = df['text_len_words'].var()
print("Variance of text lengths (in words): {:.2f}".format(var_words))
print()

# find the smallest and longest texts
smallest_text = df.loc[df['text_len_chars'].idxmin(), 'text']
smallest_text_word_len = df.loc[df['text_len_chars'].idxmin(), 'text_len_words']
longest_text = df.loc[df['text_len_chars'].idxmax(), 'text']
longest_text_word_len = df.loc[df['text_len_chars'].idxmax(), 'text_len_words']

print("Smallest text:\n", smallest_text)
print("Word length of smallest text:", smallest_text_word_len)
print("\nLongest text:\n", longest_text)
print("Word length of longest text:", longest_text_word_len)

# Task 7: Draw Wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Source for tutorial: https://www.the-analytics.club/word-cloud-python-example

# Step 1: Separate dataframe into a pos and neg dataframe
pos_df = df[df['sentiment'] == 'pos']
neg_df = df[df['sentiment'] == 'neg']

# Step 2: Concatenate all positive and negative reviews into 1 string (all_pos_text, all_neg_text)
all_pos_text = pos_df['text'].str.cat(sep=' ')
all_neg_text = neg_df['text'].str.cat(sep=' ')

# Step 3: Generate wordcloud
pos_wordcloud = WordCloud().generate(all_pos_text)
neg_wordcloud = WordCloud().generate(all_neg_text)

# Step 4: Display generated wordclouds
plt.imshow(pos_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Positive Reviews')
plt.figure(figsize=(380,380))
plt.show()

plt.imshow(neg_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Negative Reviews')
plt.figure(figsize=(380,380))
plt.show()